In [275]:
import pyreadr
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import datetime

In [276]:
# Read csv of ETF log return and Fama French Data
etf_ff_data = pd.read_csv('/Users/Ravi/Desktop/FE 630/FE-630-Final-Project/ETF_ff_Data.csv')
# Format Date column properly
etf_ff_data = etf_ff_data.rename(columns = {"Unnamed: 0":"Date"})
etf_ff_data["Date"] = pd.to_datetime(etf_ff_data["Date"])

# Print first 5 rows
print(etf_ff_data.head())

        Date       FXE       EWJ       GLD       QQQ       SPY       SHV  \
0 2007-03-01 -0.839828 -1.195132 -2.501816 -1.163088 -0.751041 -0.006970   
1 2007-03-02  0.171981 -4.117049 -8.078399 -3.797862 -3.299941  0.138968   
2 2007-03-05 -1.909022 -3.662292 -3.085226 -1.957486 -2.398792 -0.046303   
3 2007-03-06  0.673292  6.016801  4.885436  4.184908  4.311590  0.023115   
4 2007-03-07  1.112738 -0.518451  0.589248 -0.999778 -0.252521  0.092518   

        DBA       USO       XBI        ILF        EPP       FEZ  Mkt.RF   SMB  \
0 -3.359973  0.295435 -4.080605  -2.704520  -4.252163 -3.696786   -0.25 -0.11   
1 -2.364891 -1.131163 -6.168434  -5.192701  -2.743254 -3.324511   -1.24 -0.64   
2 -0.477437 -6.866887 -2.234872  -5.769796  -7.753573 -4.524134   -1.11 -0.69   
3 -1.339417  3.555023  3.079937  12.416449  10.339075  7.155172    1.58  0.65   
4  2.308413  4.857716 -0.978065  -2.114237  -0.862710 -0.476527   -0.21 -0.09   

    HML     RF  
0  0.15  0.019  
1  0.26  0.019  
2 -0.

In [277]:
def calculate_params(etf_ff_data = etf_ff_data, start = '2007-03-01', end = '2022-10-31'):
    # Subsetting data into start and end timeframe
    etf_ff_data = etf_ff_data[(etf_ff_data['Date'] >= start) & (etf_ff_data['Date'] <= end)]
    # Calculating Mkt from Fama French Factors
    mkt = etf_ff_data['Mkt.RF'] + etf_ff_data['RF']
    # Splitting df into ETF and Fama French datasets
    ff_data = etf_ff_data[["Mkt.RF", "SMB", "HML", "RF"]]
    ETF_data = etf_ff_data[['FXE', 'EWJ', 'GLD', 'QQQ', 'SPY', 'SHV', 'DBA', 'USO', 'XBI', 'ILF', 'EPP', 'FEZ']]

    # Regressing ETFs onto Fama French Factors
    reg_info = pd.DataFrame() # Data frame of coefficients returned by regressions
    ff_returns = pd.DataFrame() # Time series of returns based off of Fama French coefficients
    for i in ETF_data.columns:
        regression = LinearRegression()
        regression.fit(ff_data[["Mkt.RF", "SMB", "HML"]], ETF_data[i])
        coefficients = regression.coef_
        intercept = regression.intercept_
        ff_returns[i] = intercept + coefficients[0]*ff_data['Mkt.RF'] + coefficients[1]*ff_data['SMB'] + coefficients[2]*ff_data['HML'] + ff_data['RF']
        reg_info[i] = [intercept, coefficients[0], coefficients[1], coefficients[2]]
    reg_info = reg_info.T

    # Expected Returns (Rho)
    rho = ff_returns.mean() # ColMeans of ff_returns data frame

    # Covariance (Sigma)
    cov_returns = ff_returns.cov() # Calculated as cov of Fama French Returns dataframe
    cov_ff = np.dot(np.dot(reg_info.iloc[:, 1:4], ff_data.iloc[:, 0:3].cov()), reg_info.iloc[:, 1:4].T) # Calculated as coefficients * cov of Fama French factors * t(coefficients)

    # Beta
    beta_rm = (pd.concat([ETF_data, mkt], axis = 1).cov().iloc[:, 12]/mkt.var()).iloc[0:12] # Beta calculated with RM (Fama-French Factors) as benchmark
    beta_spy = (pd.concat([ETF_data, ETF_data['SPY']], axis = 1).cov().iloc[:, 12]/ETF_data['SPY'].var()).iloc[0:12] # Beta calculated w/ SPY as benchmark

    # Return Expected Returns (rho), Covariance (sigma), beta, and returns based off of Fama-French coefficients (for calculation of FEV)
    return(rho, cov_returns, beta_spy, ff_returns)


In [278]:
#rho, sigma, beta = calculate_params(start = '2014-01-01', end = '2020-01-01')
rho, sigma, beta, hist_returns = calculate_params()
#print(rho)
#print(sigma)
#print(beta)
print(hist_returns)

           FXE       EWJ       GLD       QQQ       SPY       SHV       DBA  \
0    -0.065679 -0.487493  0.001615 -0.743689 -0.554804  0.029784 -0.131930   
1    -0.315984 -2.288711 -0.177624 -3.445455 -2.824907  0.036715 -0.700418   
2    -0.301255 -2.042837  0.000640 -2.532407 -2.484618  0.035227 -0.701647   
3     0.400608  2.902138  0.312433  4.243852  3.718121  0.016231  0.928340   
4    -0.057847 -0.417008  0.027743 -0.570154 -0.462537  0.029479 -0.118187   
...        ...       ...       ...       ...       ...       ...       ...   
3942  0.384833  3.039275  0.797274  6.158789  3.962894  0.008407  0.813773   
3943 -0.161313 -1.413580 -0.244352 -2.639486 -1.765167  0.027428 -0.293408   
3944 -0.122730 -1.143864 -0.283761 -2.472108 -1.432807  0.026082 -0.195083   
3945  0.586032  4.540929  0.454674  6.643678  5.811159 -0.000822  1.341046   
3946 -0.168512 -1.430932 -0.206578 -2.525654 -1.779381  0.027370 -0.318547   

           USO        XBI       ILF       EPP       FEZ  
0    

In [279]:
def find_optimal_strategy_1(rho, sigma, beta, _lambda):
    w = np.asarray([1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12])
    def strategy_1(w):
        return(-1 * float(rho.T @ w - _lambda * math.sqrt(w.T @ sigma @ w)))
    constraints_1 = ({"type": "eq", "fun": lambda w: 1-sum(w)},
                {"type": "ineq", "fun": lambda w: beta.T@w+0.5},
                {"type": "ineq", "fun": lambda w: -beta.T@w+0.5},
                {"type": "ineq", "fun": lambda w: 2+min(w)},
                {"type": "ineq", "fun": lambda w: 2-max(w)})
    minim = minimize(strategy_1, w, constraints = constraints_1)
    return(minim.x)

rho, sigma, beta, hist_returns = calculate_params(start = '2007-01-01', end = '2020-01-01')
_lambda = 0.1
optimal_strat1 = find_optimal_strategy_1(rho, sigma, beta, _lambda)

# Optimal Weights
print(optimal_strat1)

# Return of Optimal Weights
print(float(rho.T @ optimal_strat1))

# Variance of Optimal Weights
print(float(optimal_strat1.T @ sigma @ optimal_strat1))

# Beta of Optimal Weights
print(np.dot(beta, optimal_strat1))

[-1.10821004 -2.          1.99999813  2.          2.00001928  2.00028133
 -2.01839291 -2.01840492 -0.09885313  0.26183758  1.99999167 -2.01826699]
0.8226782562736246
5.656243366031468
0.16869114451186062


In [280]:
def find_optimal_strategy_2(rho, sigma, beta, _lambda):
    w = np.asarray([1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12])
    TEV = np.sqrt(w.T @ sigma @ w - 2 * w.T @ hist_returns.cov()['SPY'] + hist_returns['SPY'].var())
    def strategy_2(w):
        return(-1 * float((rho.T @ w)/TEV - _lambda * math.sqrt(w.T @ sigma @ w)))
    constraints_2 = ({"type": "eq", "fun": lambda w: 1-sum(w)},
                {"type": "ineq", "fun": lambda w: beta.T@w+1},
                {"type": "ineq", "fun": lambda w: -beta.T@w+2},
                {"type": "ineq", "fun": lambda w: 2+min(w)},
                {"type": "ineq", "fun": lambda w: -1*max(w)+2})
    minim = minimize(strategy_2, w, constraints = constraints_2)
    return(minim.x)

rho, sigma, beta, hist_returns = calculate_params(start = '2014-01-01', end = '2020-01-01')
_lambda = 0.1
optimal_strat2 = find_optimal_strategy_2(rho, sigma, beta, _lambda)

# Optimal Weights
print(optimal_strat2)

# Return of Optimal Weights
print(float(rho.T @ optimal_strat2))

# Variance of Optimal Weights
print(float(optimal_strat2.T @ sigma @ optimal_strat2))

# Beta of Optimal Weights
print(np.dot(beta, optimal_strat2))

[-1.99893256  1.99104693  1.99100489  1.9967315   2.          1.9819086
 -1.99965551 -2.          0.11248255  0.9234281  -1.99988209 -1.99813242]
1.0991413797834153
35.6028817762554
1.3125178655207848
